In [ ]:
if "google.colab" in str(get_ipython()):
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


In [ ]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

In [ ]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


In [ ]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    # raw_df.printSchema()
    raw_df.show()
    return raw_df.rdd

raw_review_RDD = from_json_to_RDD(review_filename)
raw_user_RDD = from_json_to_RDD(user_filename)
raw_business_RDD = from_json_to_RDD(business_filename)


In [ ]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),    # 0
            str(entry["user_id"]),      # 1
            str(entry["business_id"]),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')


In [ ]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (str(entry["user_id"]),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


In [ ]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (str(entry["business_id"]),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD.map(get_business_tuple)

business_count = business_RDD.count()

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{usefull} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ usefull} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [ ]:
# TODO fix all comment

#review_RDD.filter(lambda x: x[3]==0).count()


#stars - useful - funny - cool - numero fan


#if stars <3
    #stars - {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2
#else
    #stars + {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2

    #dati mancanti: numero fan; best fan; best useful/funny/cool per ristorante
#rdd_test_ufc = (id ristorsante, best useful, best funny, best cool)
#best fan = query su dataset utenti --> user_RDD.max().first()

#review_id - user_id - id_rist - overall


In [ ]:
# get_overall_rating: function that generates the bias in order to incentivate or decrease the importance or overall rating of a review

# TODO fix function

# def get_overall_rating(stars, best_useful, best_funny, best_cool, useful, funny, cool, fans):
#     overall = ((useful + (funny + cool)/2) / (best_useful + (best_funny + best_cool)/2) + (fans/best_user_fans)) / 2
#     if stars < 3:
#         return stars - overall
#     else:
#         return stars + overall


In [ ]:
# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 
review_best_ufc_RDD = review_RDD.map(lambda x : (x[2], (x[4], x[5], x[6]))).reduceByKey(lambda x, y : tuple(max(x[i], y[i]) for i in range(len(y))))

# TODO split map and reduce
review_best_ufc_RDD.take(5)


In [ ]:
# TODO add map to turn the result in a simple tuple

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))
review_ufc_RDD = review_RDD.map(lambda x: (x[2], (x[0], x[1], x[3], x[4], x[5], x[6]))).join(review_best_ufc_RDD)
review_ufc_RDD.first()


In [ ]:
# TODO add function / rewrite after map in previous cell

partial_review_overall_RDD = review_ufc_RDD.map(lambda x: (x[1][0][0], x[1][0][1], x[0], x[1][0][2], 
                                                           (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2) / (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2) if sum(x[1][1]) != 0 else 0))
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

# ('A2q7d-CBM2-81tVkmS4JMw',(('RB8UpF_kT2xoOC51OzXEeA', 'EZjT2qJN0mOXypMAqZdSrQ', 2, 1, 1, 0),(21, 13, 18)))

#if(best_useful && best_funny && best_cool !=0 )
# {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fans/best user fans)}/2

# x = 10 if sum(x[1][1]) != 0 else 0


In [ ]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# TODO add map to turn the result in a simple tuple
# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


In [ ]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print(best_user_fans)

# TODO if best user fans = 0
# TODO add function / rewrite after map in previous cell
review_overall_RDD = partial_overall_fans_RDD.map(lambda x: (x[1][0][0], x[0], x[1][0][1], x[1][0][2] + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if x[1][0][2] >=3 else -2)))

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(20)


In [ ]:
business_RDD.sortBy(lambda x: x[5], ascending=False).take(10)

# business_RDD.stats()
